In [1]:
import numpy as np
import matplotlib.pyplot as plt

# 1.Регуляризация

Метод регуляризации заключается в "штрафовании" модели за слишком большие веса путем добавления нового члена к среднеквадратичной ошибке:

$$Q(w, X) + \lambda ||w||^{2} \rightarrow \underset{w}{\text{min}}.$$

добавленный член $\lambda ||w||^{2}$ - _квадратичный регуляризатор_, который представляет собой $L_{2}$-норму вектора весов, то есть сумму квадратов весов $\sum^{d}_{j=1}w_{j}^{2}$, коэффицент $\lambda$ при нем - коэффициент регуляризации. Чем больше его значение, тем меньшая сложность модели будет получаться в процессе такого обучения. Если увеличивать его, в какой-то момент оптимальным для модели окажется зануление всех весов. В то же время при слишком низких его значениях появляется вероятность чрезмерного усложнения модели и переобучения. Выбор оптимального значения этого коэфициента является отдельной задачей и заключается в многократном обучении модели с разными его значениями и сравнении их качества.

По сути, смысл регуляризации заключается, как и в обычном обучении, в минимизации функционала ошибки, только в данном случае добавляется условие **непревышения нормой вектора весов** некоторого значения $||w||^{2}\leq C$, то есть ограничение весов, что и будет залогом избежания переобучения.

Описанный выше метод с использованием $L_{2}$-нормы вектора весов в качестве регуляризатора называется _$L_{2}$-регуляризацией_. По аналогии существует также _$L_{1}$-регуляризация_, использующая в качестве регуляризатора $L_{1}$-норму вектора весов, то есть сумму модулей весов.

$$||w||_{1} = \sum^{d}_{j=1}|w_{j}|.$$

$L_{2}$-регуляризатор представляет собой **непрерывную гладкую функцию**, поэтому его добавление не усложняет использование градиентных методов оптимизации, так как в каждой его точке существует производная. $L_{1}$-регуляризатор уже не является гладкой функцией, так как в нем есть модуль, у которого не существует производной в нуле. То есть его использование усложняет градиентные методы оптимизации, если необходимо будет искать производную в нуле. Однако, L1 регуляризация обладает интересной особенностью, заключающейся в занулении некоторых весов при его применении. Иными словами, происходит отбор признаков, в результате чего остаются только самые важные признаки. Обычно сначала подбирают порядок регуляризации(e^-1, e^-2, и.т.д). Далее
параметр можно подбирать более точнее. Далеко не всегда линейная модель чувствительна к гиперпараметрам, поэтому можно ограничиться только подбором порядка регуляризации.

# 2. Стохастический градиентный спуск

Если расписать $j$-ю компонетну градиентного спуска, то получим

$$\frac{\partial Q}{\partial w_{j}} = \frac{2}{l}\sum^{l}_{i=1}x^{j}_{i}(\left \langle w,x_{i} \right \rangle - y_{i}),$$

то есть суммирование по всем $l$ объектам обучающей выборки. Здесь выражение под суммой показывает, как нужно изменить $j$-й вес, чтобы как можно сильнее улучшить качество __на объекте $x_{i}$__, а вся сумма показывает, как нужно изменить вес, чтобы улучшить качество на __всей выборке__.

В этой формуле отражен один из главных недостатков градиентного спуска: если выборка большая по объему, то даже один шаг градиентного спуска будет занимать много вычислительных ресурсов и времени.

Стремление к оптимизации процесса привело к появлению _стохастического градиентного спуска_ (Stochastic gradient descent, SGD). Идея его основана на том, что на одной итерации мы вычитаем не вектор градиента, вычисленный по всей выборке, а вместо этого случайно выбираем один объект из обучающей выборки $x_{i}$ и вычисляем градиент только на этом объекте, то есть градиент только одного слагаемого в функционале ошибки и вычитаем именно этот градиент из текущего приближения вектора весов:

$$w^{k} = w^{k-1} - \eta_{k}\nabla Q(w^{k-1}, \{x_{i}\}),$$

то есть $\nabla Q(w^{k-1}, X)$ заменяется на $\nabla Q(w^{k-1}, \{x_{i}\})$.

Если в случае градиентного спуска мы стараемся на каждой итерации уменьшить ошибку на всей выборке, и по мере увеличения числа итераций ошибка падает монотонно, то в случае стохастического градиентного спуска мы уменьшаем на каждой итерации ошибку **только на одном либо нескольких объектах**, но при этом есть вероятность увеличить ее на другом объекте, поэтому график изменения ошибки может получаться немонотонным, и даже иметь пики. То есть на какой-то итерации мы можем даже увеличить ошибку, но при этом в целом по ходу метода ошибка снижается, и рано или поздно мы выходим на нормальный уровень. Также, как и в обычном 
градиентом спуске, в итоге находим оптимальные значения весов с использованием информации на предыдущей итерации.

# 3. Примеры

## 3.1 Сравнение градиентного спуска и стохастического градиентного спуска

Сгенерировать датасет при помощи sklearn.datasets.make_regression и обучить линейную модель при помощи градиентного и стохастического градиентного спуска. Нанести среднеквадратичную ошибку для обоих методов на один график, сделать выводы о разнице скорости сходимости каждого из методов.

In [3]:
from sklearn.datasets import make_regression

In [123]:
n_samples = 1000
n_features = 3
n_informative= 2
n_targets = 1
noise = 5
rng = np.random.RandomState(42)
X, y, coef = make_regression(n_samples, n_features, n_informative, n_targets, 
                             noise, coef = True, random_state = rng)

In [66]:
# реализуем функцию, определяющую среднеквадратичную ошибку
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

Градиентный спуск

In [116]:
def eval_gd_model(X, y, iterations, alpha=1e-4):
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = np.dot(W, X)
        err = calc_mse(y, y_pred)
        W -= (alpha * (1/n * 2 * np.dot((y_pred - y), X.T)))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [124]:
eval_model(X, y, iterations=1000, alpha=1e-2)

ValueError: operands could not be broadcast together with shapes (1000,) (3,) 

Стохастический градиентный спуск

In [114]:
def eval_sgd_model(X, y, iterations=None, batch_size=None, alpha=None):
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    n_batch = n // batch_size #количество элементов в 'порции'
    if n % batch_size != 0: #если количество элементов нечётное
        n_batch += 1
    for i in range(1, iterations+1):
        for b in range(n_batch):
            start_ = batch_size*b #вычисляем индексы объектов, которые должны попасть
            end_ = batch_size*(b+1) #вычисляем индексы объектов, которые должны попасть

            # print(b, n_batch, start_, end_)

            X_tmp = X[:, start_ : end_] #выбор элемента
            y_tmp = y[start_ : end_] #выбор элемента
            y_pred_tmp = np.dot(W, X_tmp)
            err = calc_mse(y_tmp, y_pred_tmp)
            W -= (alpha * (1/n * 2 * np.dot((y_pred_tmp - y_tmp), X_tmp.T)))

    if i % (iterations / 50) == 0:
        print(i, W, err)

In [112]:
X.shape[0]

1000

In [111]:
eval_gd_model(X, y, iterations=1000, alpha=1e-4)

ValueError: shapes (1000,3) and (1000,) not aligned: 3 (dim 1) != 1000 (dim 0)

In [115]:
eval_sgd_model(X, y, iterations=5000, batch_size=1, alpha=1e-4)

5000 [-9.12339632e-01 -1.57554610e+00 -1.60052691e+00  1.07307241e+00
 -1.60323961e+00 -5.85304850e-01  5.75331098e-01  7.13954268e-01
 -1.63164925e+00 -8.63014522e-01  3.51989917e-01 -4.07495381e-01
 -6.63993370e-01  4.54802731e-01 -1.18359314e+00 -1.23874776e+00
 -7.65467493e-01  4.14896369e-01  1.44153620e+00 -5.41778242e-02
 -9.79022386e-01 -8.47063470e-01  8.08303122e-01 -1.90510472e+00
  7.68833856e-01  5.15652604e-01  6.83322472e-01  9.88097123e-01
  1.42699720e+00  1.43023562e+00 -2.09106847e-01 -7.94665296e-01
  1.29899810e+00 -1.92294196e-02  5.10643129e-01 -9.11002204e-01
  1.03600604e+00 -8.84327251e-01 -3.08592888e-01 -5.49355261e-01
  6.21820149e-02  8.95133580e-01 -3.25138256e-01 -6.84829812e-01
 -4.83399733e-01 -1.80591751e+00 -8.78506492e-01  1.95832552e+00
 -8.94896025e-01  3.76557277e-01 -6.72952275e-01 -3.89897764e-01
 -8.80498234e-01 -3.72665854e-01 -2.27429466e-01  1.01593790e+00
  7.55084117e-01  5.99104098e-01  5.95298512e-01 -7.67239824e-01
  1.41899348e+00 -6.

Batch_size обычно выбирают как 2 в степени общего числа объектов. Чем он меньше тем лучше, поскольку чем чаще будем обновлять веса, тем быстрее добьёмся сходимости.

![title](batch.png)

## 3.2. Модифицировать решение первого задания путем добавления 𝐿2 -регуляризации и сравнить результаты.

In [148]:
def eval_gd_model_reg2(X, y, iterations=None, alpha=None, lambda_= None):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = np.dot(W, X)
        err = calc_mse(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + 2 * lambda_* W) #производная от двух слагаемых равна 
                                                                    #производной каждого слагаемого
    if i % (iterations / 10) == 0:
        print(i, W, err)

In [149]:
eval_gd_model_reg2(X, y, iterations=5000, alpha=1e-2, lambda_=1e-4)

ValueError: operands could not be broadcast together with shapes (1000,) (3,) 

In [150]:
def eval_sgd_model_reg2(X, y, iterations=None, batch_size=None, alpha=None, lambda_=None):
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    n_batch = n // batch_size #количество элементов 
    if n % batch_size != 0: #если количество элементов нечётное
        n_batch += 1
    for i in range(1, iterations+1):
        for b in range(n_batch):
            start_ = batch_size*b #вычисляем индексы объектов, которые должны попасть
            end_ = batch_size*(b+1) #вычисляем индексы объектов, которые должны попасть

            # print(b, n_batch, start_, end_)

            X_tmp = X[:, start_ : end_] #выбор элемента
            y_tmp = y[start_ : end_] #выбор элемента
            y_pred_tmp = np.dot(W, X_tmp)
            err = calc_mse(y_tmp, y_pred_tmp)
            W -= (alpha * (1/n * 2 * np.dot((y_pred_tmp - y_tmp), X_tmp.T)) + 2 * lambda_* W)

    if i % (iterations / 50) == 0:
        print(i, W, err)

In [151]:
eval_sgd_model_reg2(X, y, iterations=5000, batch_size = 1, alpha=1e-2, lambda_=1e-4)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in square
  This is separate from the ipykernel package so we can avoid doing imports until


5000 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan n